# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import scipy 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print('Данные для Нулевого региона:')
print('Количество дубликатов:', data_0.duplicated(subset='id').sum(), '\n')
display(data_0.info())
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print('\nДанные для Первого региона:')
print('Количество дубликатов:', data_1.duplicated(subset='id').sum(), '\n')
display(data_1.info())
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print('\nДанные для Второго региона:')
print('Количество дубликатов:', data_2.duplicated(subset='id').sum(), '\n')
display(data_2.info())

Данные для нулевой скважины:
Количество дубликатов: 10 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None


Данные для первой скважины:
Количество дубликатов: 4 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None


Данные для второй скважины:
Количество дубликатов: 4 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Данные имеют правильный типа, пропусков не обнаружено.  
Есть повторяющиеся значения в столбце с уникальным идектификатором скважины *id*, эти строки лучше удалить.

In [3]:
data_0 = data_0.drop_duplicates(subset='id')
data_1 = data_1.drop_duplicates(subset='id')
data_2 = data_2.drop_duplicates(subset='id')

In [14]:
# проверка
data_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99990 non-null  object 
 1   f0       99990 non-null  float64
 2   f1       99990 non-null  float64
 3   f2       99990 non-null  float64
 4   product  99990 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


## Обучение и проверка модели

### Разделение данных на выборки

In [5]:
features_0 = data_0.drop(['product', 'id'], axis=1)
target_0 = data_0['product']

features_1 = data_1.drop(['product', 'id'], axis=1)
target_1 = data_1['product']

features_2 = data_2.drop(['product', 'id'], axis=1)
target_2 = data_2['product']

features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
features_0, target_0, test_size=0.25, random_state=123)
print(f'0. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные)',
     features_0_train.shape, features_0_valid.shape, target_0_train.shape, target_0_valid.shape)

features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
features_1, target_1, test_size=0.25, random_state=123)
print(f'1. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные)',
     features_1_train.shape, features_1_valid.shape, target_1_train.shape, target_1_valid.shape)

features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
features_2, target_2, test_size=0.25, random_state=123)
print(f'2. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные)',
     features_2_train.shape, features_2_valid.shape, target_2_train.shape, target_2_valid.shape)

0. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные) (74992, 3) (24998, 3) (74992,) (24998,)
1. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные) (74997, 3) (24999, 3) (74997,) (24999,)
2. Признаки (тренировочные, валидационные); таргеты (тренировочные, валидационные) (74997, 3) (24999, 3) (74997,) (24999,)


Данные для трех регионов разбиты на тренировочную и валидационную выборки в соотношении 75:25.

### Обучение моделей линейной регрессии

In [6]:
model_0 = LinearRegression()
model_0.fit(features_0_train, target_0_train)
predicted_valid_0 = model_0.predict(features_0_valid)
true_answer_0 = target_0_valid
rmse_0 = mean_squared_error(true_answer_0, predicted_valid_0, squared=False)
print('Нулевая модель')
print(f'Среднее значение {predicted_valid_0.mean():.2f}, RMSE {rmse_0:.2F}')
print(f'Коэффициент детерминации R2: {r2_score(target_0_valid, predicted_valid_0):.2f}')

Нулевая модель
Среднее значение 92.68, RMSE 37.60
Коэффициент детерминации R2: 0.27


In [7]:
model_1 = LinearRegression()
model_1.fit(features_1_train, target_1_train)
predicted_valid_1 = model_1.predict(features_1_valid)
true_answer_1 = target_1_valid
rmse_1 = mean_squared_error(true_answer_1, predicted_valid_1, squared=False)
print('Первая модель')
print(f'Среднее значение {predicted_valid_1.mean():.2f}, RMSE {rmse_1:.2F}')
print(f'Коэффициент детерминации R2: {r2_score(target_1_valid, predicted_valid_1):.2f}')

Первая модель
Среднее значение 69.31, RMSE 0.89
Коэффициент детерминации R2: 1.00


In [8]:
model_2 = LinearRegression()
model_2.fit(features_2_train, target_2_train)
predicted_valid_2 = model_2.predict(features_2_valid)
true_answer_2 = target_2_valid
rmse_2 = mean_squared_error(true_answer_2, predicted_valid_2, squared=False)
print('Вторая модель')
print(f'Среднее значение {predicted_valid_2.mean():.2f}, RMSE {rmse_2:.2F}')
print(f'Коэффициент детерминации R2: {r2_score(target_2_valid, predicted_valid_2):.2f}')

Вторая модель
Среднее значение 94.94, RMSE 40.05
Коэффициент детерминации R2: 0.20


**Выводы**  
Модели для Нулевого и Второго регионов далеки от идеала, об этом говорит высокое значение квадрата из среднеквадратичной ошибки и значение коэффициента детерминации, близкое к нулю. Возможно, в данных присутствует много выбросов, и модели не удалось установить связь между характеристиками месторождений и количеством запасов нефти в них.  
Модель Первого региона делает прогнозы с практически стопроцентной точностью, однако среднее значение ниже среднего для Нулевого и Второго регионов.  
Делать выводы о выборе региона пока нельзя, нужно понять, где добыча с большей вероятностью будет более прибыльной.

## Подготовка к расчёту прибыли

In [9]:
# Бюджет
budget = 10e9
# Количество скважин под разработку
quantity = 200
# Цена за баррель, за у.е. (тысячу баррелей)
price_barrel = 450
price_unit = 450e3

# Количество тысяч баррелей в скважине для безубыточной разработки
product_min = (budget/quantity)/price_unit
print(f'Количество баррелей, тыс. для безубыточной разработки одной скважины {product_min:.2f}')

Количество баррелей, тыс. для безубыточной разработки одной скважины 111.11


Это значение выше, чем средние значения "урожайности" скважин для всех трех регионов.  
Учитывая неточные модели и разброс запасов нефти для Нулевого и Второго регионов, нельзя делать выводы о прибыльности региона лишь на основе сравнения средних значений запасов и минимально необходимого объема нефти.

## Расчёт прибыли и рисков 

### Расчет прибыли для лучших предсказанных скважин

Рассчитаем для каждого региона прибыль для 200 лучших скважин, предсказанных моделями.

In [10]:
best_0 = pd.Series(predicted_valid_0).sort_values(ascending=False).head(200)
profit_0 = best_0.sum()*price_unit - budget
print(f'Прибыль в Нулевом регионе {profit_0:.2e} рублей')

best_1 = pd.Series(predicted_valid_1).sort_values(ascending=False).head(200)
profit_1 = best_1.sum()*price_unit - budget
print(f'Прибыль в Первом регионе {profit_1:.2e} рублей')

best_2 = pd.Series(predicted_valid_2).sort_values(ascending=False).head(200)
profit_2 = best_2.sum()*price_unit - budget
print(f'Прибыль во Втором регионе {profit_2:.2e} рублей')

Прибыль в Нулевом регионе 3.90e+09 рублей
Прибыль в Первом регионе 2.49e+09 рублей
Прибыль во Втором регионе 3.42e+09 рублей


Ожидаемо, в Первом регионе с наименьшим средним значением прибыль с двухсот лучших скважин гораздо меньше, 2,49 миллиарда против 3,9 и 3,42 в Нулевом и Втором.

### Расчет распределения прибыли и вероятность убытков

Рассчитаем среднюю прибыль и 95% доверительный интервал, а также вероятность убытков.

In [11]:
# фунция для расчета прибыли
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return price_unit * selected.sum() - budget

state = np.random.RandomState(123)

In [12]:
target_0_valid = pd.Series(target_0_valid)
predicted_valid_0 = pd.Series(predicted_valid_0, index=target_0_valid.index)

values = []
for i in range(1000):
    target_subsample = target_0_valid.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_0[target_subsample.index]  
    values.append(revenue(target_subsample, probs_subsample, quantity))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print(f"Нулевой регион\nСредняя прибыль: {mean:.2e}")
print(f"95% интервал: от {lower:.2e} до {upper:.2e}")
print(f"Вероятность убытков: {scipy.stats.percentileofscore(values, 0):.1f}%")

target_1_valid = pd.Series(target_1_valid)
predicted_valid_1 = pd.Series(predicted_valid_1, index=target_1_valid.index)

values = []
for i in range(1000):
    target_subsample = target_1_valid.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_1[target_subsample.index]  
    values.append(revenue(target_subsample, probs_subsample, quantity))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print(f"\nПервый регион\nСредняя прибыль: {mean:.2e}")
print(f"95% интервал: от {lower:.2e} до {upper:.2e}")
print(f"Вероятность убытков: {scipy.stats.percentileofscore(values, 0):.1f}%")

target_2_valid = pd.Series(target_2_valid)
predicted_valid_2 = pd.Series(predicted_valid_2, index=target_2_valid.index)

values = []
for i in range(1000):
    target_subsample = target_2_valid.sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid_2[target_subsample.index]  
    values.append(revenue(target_subsample, probs_subsample, quantity))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print(f"\nВторой регион\nСредняя прибыль: {mean:.2e}")
print(f"95% интервал: от {lower:.2e} до {upper:.2e}")
print(f"Вероятность убытков: {scipy.stats.percentileofscore(values, 0):.1f}%")

Нулевой регион
Средняя выручка: 4.24e+08
95% интервал: от -1.03e+08 до 9.43e+08
Вероятность убытков: 6.1%

Первый регион
Средняя выручка: 5.36e+08
95% интервал: от 1.07e+08 до 9.92e+08
Вероятность убытков: 0.6%

Второй регион
Средняя выручка: 3.65e+08
95% интервал: от -1.94e+08 до 9.38e+08
Вероятность убытков: 11.1%


**Вывод**  

Наиболее перспективным является Первый регион: по прогнозу там ожидается самая высокая средняя прибыль, вероятность убытков практически равна нулю.  
Для Нулевого и Второго регионов вероятность убытков превышает отметку в 2.5%, средняя прибыль там меньше, чем в Первом регионе, поэтому их не стоит рассматривать в качестве варианта для разработки месторождений.